Recording is available here: https://www.youtube.com/watch?v=eHayF4sDTX8
***italicized text***

# **Question 1**

# *סעיף א*

In [2]:
import numpy as np
import torch
def expand_a_as_b(a, b):
    """
        broadcasting a as b
    """
    # lists of tensors a and b containing how much elements we have in each dimension
    a_dimensions_list = list(a.shape)
    b_dimensions_list = list(b.shape)
    # How much dimensions does each tensor have
    a_dim = len(a_dimensions_list)
    b_dim = len(b_dimensions_list)

    c = torch.clone(a)

    if a_dim > b_dim:
        raise RuntimeError("The expanded sizes do not match")
    elif a_dim < b_dim:
        c = c.reshape((b_dim - a_dim) * [1] + a_dimensions_list)

    # The expanded dimensions of tensor a, after appending ones to it at the beginning
    c_dimensions_list = list(c.shape)

    # Testing operation validity
    val_flag = True
    for i in range(b_dim):
        true_status = (c_dimensions_list[i] == b_dimensions_list[i]) or (c_dimensions_list[i] == 1)
        if not true_status:
            val_flag = False
            break

    # Number of repeat dimensions:
    if val_flag:
        for i in range(len(c.shape)):
            if c_dimensions_list[i] != b_dimensions_list[i]:
                repeated_dim = (c,) * b_dimensions_list[i]
                c = torch.stack(repeated_dim, i)
                c = torch.squeeze(c, i+1)
        return c
    else:
        raise RuntimeError("The expanded sizes do not match")

# סעיף *ב*

In [3]:
def brodcast_both_check(a, b):
    # We assume that always a is with dimension smaller than b, if not we swap them
    # x tensor is with dimension always smaller than y tensor
    x = a
    y = b

    if b.dim() < a.dim():
        x = torch.clone(b)
        y = torch.clone(a)

    x_dimensions_list = list(x.shape)
    y_dimensions_list = list(y.shape)

    x_dimension = len(x_dimensions_list)
    y_dimension = len(y_dimensions_list)

    x = x.reshape((y_dimension - x_dimension) * [1] + x_dimensions_list)
    x_dimensions_list = list(x.shape)

    # Check that both tensors are broadcastable together
    resulting_tensor_dimensions = ()
    for k in range(y_dimension):
        if x_dimensions_list[k] == y_dimensions_list[k]:
            resulting_tensor_dimensions = resulting_tensor_dimensions + (x_dimensions_list[k],)
        elif x_dimensions_list[k] == 1:
            resulting_tensor_dimensions = resulting_tensor_dimensions + (y_dimensions_list[k],)
        elif y_dimensions_list[k] == 1:
            resulting_tensor_dimensions = resulting_tensor_dimensions + (x_dimensions_list[k],)
        else:
            return False, None
            break

    return True, resulting_tensor_dimensions

# **סעיף ג**

In [4]:
def broadcast_both(a, b):
    tensors_are_broadcastable = brodcast_both_check(a, b)[0]
    resulting_tensor_dimensions = brodcast_both_check(a, b)[1]
    if tensors_are_broadcastable:
        dummy_tensor_for_expand = torch.zeros(resulting_tensor_dimensions)
        a_expanded = expand_a_as_b(a, dummy_tensor_for_expand)
        b_expanded = expand_a_as_b(b, dummy_tensor_for_expand)
        return a_expanded, b_expanded
    else:
        raise RuntimeError("The tensors are not broadcastable together")

# **סעיף ד**

In [8]:
a = torch.empty(5,3,4,2)
b = torch.empty(3,4,1)
res_broadcast = broadcast_both(a, b)
res_broadcast_origin = torch.broadcast_tensors(a, b)
print((torch.eq(res_broadcast[0], res_broadcast_origin[0])).all())
print((torch.eq(res_broadcast[1], res_broadcast_origin[1])).all())


a = torch.arange(3).reshape(3, 1, 1)
b = torch.arange(12).reshape(3, 2, 2)
res_broadcast = broadcast_both(a, b)
res_broadcast_origin = torch.broadcast_tensors(a, b)
print((torch.eq(res_broadcast[0], res_broadcast_origin[0])).all())
print((torch.eq(res_broadcast[1], res_broadcast_origin[1])).all())

tensor(True)
tensor(True)
tensor(True)
tensor(True)
